## Tarea 1
#### Maker: Iván Paredes Reséndiz
##### Ultima actualización: 26-01-24

Resumen: En este script desarrollaremos un modelo para estimar adecuadamente el precio de las casas con diversas tecnicas de análisis. La estructura del documento es la siguiente: 1. Cargar bases y librerias, 2. Analizar relaciones, 3. Generar modelo, 4. Realizar pronosctico; y 5. Comparar modelo

In [ ]:
# Cargamos librerías, en general agregamos su documentación para futura consulta, salvo por pandas que es de uso más común.
import pandas as pd
import seaborn as sns #https://seaborn.pydata.org/installing.html
import statsmodels.api as smf #https://www.statsmodels.org/v0.10.2/importpaths.html
import statsmodels.formula.api as smq
import statsmodels.regression.linear_model as sm
from sklearn.model_selection import train_test_split #https://scikit-learn.org/stable/
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt #https://matplotlib.org/2.0.2/users/pyplot_tutorial.html

In [ ]:
# Cargar el dataset de entrenamiento que se encuentra en el mismo lugar de 
df_training = pd.read_csv('train.csv')

En toda esta sección hacemos una exploración de la data de entrenamiento que tenemos. Por ejemplo, el tamaño de la base en términos de observaciones y variables, un summary con los valores de media, varianza y otras medidas estadísticas.

In [ ]:
# Ver el tamaño del set
print("train size:",df_training.shape)

# El resultado es: train size: (1460, 81)

In [ ]:
#Vemos las 10 primeras observaciones y las variables numericas.
df_training.describe()

In [ ]:
#Información de los tipos de variable, hay bastantes que son objeto.
df_training.info()

In [ ]:
# Describimos la variable dependiente de precio de venta y 
print(df_training['SalePrice'].describe())

Hay varias cosas que observar en los datos de la variable de Precio de venta que es la que tratamos de pronosticar. Por ejemplo, tiene datos atipicos altos, que es conveniente eliminar. Esto se puede observar más claramente en el histograma,

In [ ]:
#Graficamos su histograma
plt.figure(figsize=(9, 5)) # Establecemos tamaño
sns.distplot(df_training['SalePrice'], color='r', bins=50, hist_kws={'alpha': 0.4});

In [ ]:
# Eliminamos los datos que no son numericos y creamos un mapa de calor con la relación entre las distintas variables.
df_corr = df_training.astype("float64",errors='ignore')
df_corr = df_corr.select_dtypes(exclude="object")
plt.subplots(figsize=(12,9))
sns.heatmap(df_corr.corr())

En el mapa de calor anterior podemos observar algunas de las variables más relevantes y su correlación con el Precio de venta SalePrice. Del mismo modo podemos observar que hay otras variables como el PoolArea o variables relacionadas con el portíco que parecen no tener correlación con el precio de venta.

In [ ]:
# Catalogamos las 10 caracteristicas principales
top15corrcaracteri = df_corr.corr()['SalePrice']
top15corrcaracteri = top15corrcaracteri[:15]
print(top15corrcaracteri)


In [ ]:
# Realizamos planos de scatter con variables relevates antes calculadas 
plt.title("Scatter de 'SalePrice'vs '1stFlrSF'")
plt.scatter(x=df_training['1stFlrSF'], y=df_training['SalePrice'])
plt.plot()

In [ ]:
# Realizamos planos de scatter con variables relevates antes calculadas 
plt.title("Scatter de 'SalePrice'vs 'OverallQual'")
plt.scatter(x=df_training['OverallQual'], y=df_training['SalePrice'])
plt.plot()

In [ ]:
# Realizamos planos de scatter con variables relevates antes calculadas 
plt.title("Scatter de 'SalePrice'vs 'TotalBsmtSF'")
plt.scatter(x=df_training['TotalBsmtSF'], y=df_training['SalePrice'])
plt.plot()

In [ ]:
#elminamos los valores extremos, la segmentacion se debe a que en el histograma esos valores son pocos y están por encima del 95% de la muestra
df_training.drop(df_training[(df_training['SalePrice']>450000)].index, inplace=True)

In [ ]:
# Observamos el nuevo tamaño de la base, se perdieron menos de 14 valores, el 0.001% de la base
print("train size:",df_training.shape)

In [ ]:
# Preparamos los datos de la regresión, agregamos una constante
X = df_training[['OverallQual','LotArea','GrLivArea','GarageCars','YearBuilt','YearRemodAdd','MSSubClass','1stFlrSF','TotalBsmtSF']]  # Cambia 'tu_variable_independiente' por el nombre de tu columna
y = df_training['SalePrice'] 
X = smf.add_constant(X)

In [ ]:
# Crear el modelo de regresión lineal
#modelo = LinearRegression()

### Justificación del modelo usado.
El uso de mínimos cuadrados para analizar y pronosticar el precio de venta de biene inmuebles es eficiente y fácil de interpretar. Consideró que en este caso la relación entre las variables independientes (como el tamaño de la casa, el númedo de estacionamientos, el año de renovación) y la variable dependiente (el precio de la casa) es lineal para valores no extremos. Los  mínimos cuadrados, al buscar minimizar la suma de los cuadrados de las diferencias entre los valores observados y los valores predichos, proporcionan una manera directa y objetiva de estimar los coeficientes de la relación lineal. El enfoque de mínimos cuadrados facilita la interpretación de los resultados, ya que los coeficientes estimados pueden ser directamente interpretados como el cambio en el precio de la vivienda por cada unidad de cambio en las variables independientes, asumiendo que todas las demás variables se mantienen constantes.


In [ ]:
model = sm.OLS(y,X)
fitted_model = model.fit()
fitted_model.summary()

In [ ]:
# El modelo señala que The condition number is large, 1.8e+06. This might indicate that there are strong multicollinearity or other numerical problems. 
# La solución es eliminar 1stFlrSF que puede estar repetida con TotalBsmtSF, YearBuilt y GrLivArea

In [ ]:
# Preparamos los datos de la regresión corrigiendo lo que señalamos en la línea previa
X = df_training[['OverallQual','LotArea','GarageCars','YearRemodAdd','MSSubClass','TotalBsmtSF']]  # Cambia 'tu_variable_independiente' por el nombre de tu columna
y = df_training['SalePrice'] 
X = smf.add_constant(X)
model = sm.OLS(y,X)
fitted_model = model.fit()
fitted_model.summary()

In [ ]:
# Cargar el dataset de entrenamiento
df_test = pd.read_csv('C:/Users/CFC/Downloads/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
#Observamos su información, el número de datos que tenemos es cercano al que tenemos como entrenamiento
df_test.info()

In [ ]:
#Vemos las 10 primeras observaciones y las variables numericas.
df_test.describe()

In [ ]:
X_test = df_test[['OverallQual','LotArea','GarageCars','YearRemodAdd','MSSubClass','TotalBsmtSF']]
X_test = smf.add_constant(X_test)
df_test.describe()

In [ ]:
## Con el modelo antes calculado, aquí le pedimos al usuario que señale sus valores esperados para cada variable.
numero1 = float(input(f"Del 1 al 10 en que tan buena condición esperas que este la casa, siendo 1 mala y 10 excelente: "))
numero2 = float(input(f"De qué tamaño esperas que sea el terreno en pies cuadrados: "))
numero3 = float(input(f"Cuántos lugares de estacionamiento quieres: "))
numero4 = float(input(f"Año de su última renovación en 4 cifras: "))

# Realizamos el cálculo
resultado = -8.485e+05 + (numero1 * 2.989e+04) + (numero2 * 1.1699) + (numero3 * 2.29e+04) + (numero4 * 399.0900)

# Imprimimos el resultado
print(f"El precio aproximado de una casa con esas caracteristicas es: {int(resultado)}")
